In [6]:
import csv
import pandas as pd

/var/folders/pl/pg3bvqz92xqgj0sbq31wjfqr0000gn/T/ipykernel_67896/584824882.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [59]:
def get_file_name(path):
    name = path.split("/")[-1].split(".")[0]
    return name

def make_df(file, rank_cut_off):

    # Open csv file
    with open(file, 'r') as tsv_file:
        tsv_reader = csv.reader(tsv_file, delimiter='\t')
        data = list(tsv_reader)

    gene_list = []
    cell_type_list = []
    data_dict = {}
        
    # Imports data to a gene list and cell type list
    for sample in data:
        if len(sample) == 7:
            gene_list.append(sample[1])
            cell_type_list.append(sample[5])

    # Removes duplicate genes
    gene_list = list(set(gene_list))

    # creates dictionary of dictionaries with key as cell type and gene count as value
    for cell in cell_type_list:
        data_dict[cell] = {gene: 0 for gene in gene_list}

    # Counts ranks
    for sample in data:
        if len(sample) == 7:
            if int(sample[6]) <= rank_cut_off:
                data_dict[sample[5]][sample[1]] += 1

    samples_to_delete = list(data_dict.keys())

    # Deletes cell types where expression is 0
    for sample in samples_to_delete:
        all_zero = all(value == 0 for value in data_dict[sample].values())
        if all_zero:
            del data_dict[sample]

    # Remove "unknown" cell types
    if 'Unknown' in data_dict:
        del data_dict['Unknown']

    #Saves file
    df = pd.DataFrame(data_dict).transpose()

    df_file_name = "df_cutoff" + str(rank_cut_off) + "_" + get_file_name(file) + ".csv"


    df.to_csv(df_file_name, index=True)

In [60]:
files = [
    'subcluster0',
    'subcluster1',
    'subcluster2',
    'subcluster3',
    'subcluster4',
    'subcluster5',
    'subcluster6',
    'subcluster7',
    'subcluster8'
]

for i in range(len(files)):
    files[i] = '/Users/gracewang/Documents/mesirov_lab/seurat-medullo/panglao_subclusters/' + files[i] + ".csv"

for file in  files:
    make_df(file, 20)
    make_df(file, 10)
    make_df(file, 5)
